In [ ]:
#!pip install pytorch-lightning==1.9.4
!pip install packaging==23.2
!pip install deepforest 

In [ ]:
#load the modules
import os
import time
import numpy as np
import cv2
import glob
import pandas as pd
import random
import torch
from matplotlib import pyplot as plt 
import os
from deepforest import main
from deepforest import get_data
from deepforest import utilities
from deepforest import preprocess
from deepforest import model
from sklearn.decomposition import PCA
import math
from itertools import cycle , islice


In [ ]:
def preprocess_annons(path_to_csv):
    df = pd.read_csv(path_to_csv, sep=',', names=['image_path', 'xmin', 'ymin', 'xmax', 'ymax', 'label'])
    return df

In [ ]:
config_path = get_data("deepforest_config.yml")
config = utilities.read_config(config_path)
config["num_classes"] = 16
config['train']['scheduler']['params']['eps'] = float(config['train']['scheduler']['params']['eps']) # why is this not in float?
print(type(config['train']['scheduler']['params']['eps'])) # sanity test
config["batch_size"] = 8

# ----------------- Use-FasterRCNN---------------------------- #
config['architecture'] = 'FasterRCNN'
# Update scheduler settings in the "train" section
config["train"]["scheduler"]['type'] = "cosine"


In [ ]:
DIR="/kaggle/input/gs-attempt2/GS-REMAPPED"
label_dict = {i: i for i in range(16)}

in_train_annots = f"{DIR}/train/_annotations.csv"
in_test_annots =  f"{DIR}/test/_annotations.csv"
in_valid_annots = f"{DIR}/valid/_annotations.csv"

os.mkdir("/kaggle/working/train/")
os.mkdir("/kaggle/working/test/")
os.mkdir("/kaggle/working/valid/")


out_train_annotations="/kaggle/working/train/_annotations.csv"
out_test_annotations="/kaggle/working/test/_annotations.csv"
out_valid_annotations="/kaggle/working/valid/_annotations.csv"


preprocess_annons(in_train_annots).to_csv(out_train_annotations)
preprocess_annons(in_test_annots).to_csv(out_test_annotations)
preprocess_annons(in_valid_annots).to_csv(out_valid_annotations)

m = main.deepforest(config_args=config, label_dict=label_dict)
m.config["train"]["epochs"] = 100
m.config["train"]["csv_file"] = out_train_annotations 
m.config["train"]["root_dir"] = os.path.dirname(in_train_annots)

    
m.config["validation"]["csv_file"] = out_valid_annotations 
m.config["validation"]["root_dir"] = os.path.dirname(in_valid_annots)
m.config["validation"]["val_accuracy_interval"] = 1

try:
    os.mkdir("/kaggle/working/checkpoints")
except FileExistsError:
    pass
m.create_trainer()

In [ ]:
m.trainer.fit(m)

In [ ]:
#save the prediction result to a prediction folder
save_dir = "/kaggle/working/prediction_data"

try:
    os.mkdir(save_dir)
except FileExistsError:
    pass
results = m.evaluate(out_test_annotations, os.path.dirname(in_test_annots), iou_threshold = 0.5, savedir = save_dir)

In [ ]:
results

In [ ]:
import numpy as np
import pandas as pd

def compute_ap(recall, precision):
    """Compute AP using the trapezoidal rule."""
    recall = np.concatenate(([0], recall, [1]))  # Add 0 and 1 for completeness
    precision = np.concatenate(([1], precision, [0]))  # Ensure precision curve is well-formed
    
    for i in range(len(precision) - 2, -1, -1):
        precision[i] = max(precision[i], precision[i + 1])  # Make precision non-decreasing
    
    indices = np.where(recall[1:] != recall[:-1])[0]  # Indices where recall changes
    ap = np.sum((recall[indices + 1] - recall[indices]) * precision[indices + 1])
    return ap

def compute_map(results, iou_threshold=0.5):
    """Compute mAP@50 given evaluation results."""
    classes = results['true_label'].unique()
    aps = []

    for cls in classes:
        cls_results = results[results['true_label'] == cls]  # Get predictions for this class
        cls_results = cls_results.sort_values(by='score', ascending=False)

        tp = (cls_results['IoU'] >= iou_threshold) & (cls_results['match'])
        fp = ~tp

        tp_cumsum = np.cumsum(tp)
        fp_cumsum = np.cumsum(fp)

        recall = tp_cumsum / max(1, (results['true_label'] == cls).sum())  # Avoid division by zero
        precision = tp_cumsum / (tp_cumsum + fp_cumsum)

        ap = compute_ap(recall, precision)
        aps.append(ap)
        print(f"class {cls} ------ {ap}")

    return np.mean(aps) if aps else 0.0  # Compute mean AP

# Compute mAP@50
map_50 = compute_map(results['results'], iou_threshold=0.5)
print(f"mAP@50: {map_50:.4f}")
